# Movies Database: Data Cleaning

by Israel Diaz

## Data Description

The data correspond to the one downloaded from [IMDB source](https://datasets.imdbws.com/).

**IMDb Dataset Details**

Each dataset is contained in a gzipped, tab-separated-values (TSV) formatted file in the UTF-8 character set. The first line in each file contains headers that describe what is in each column. A ‘\N’ is used to denote that a particular field is missing or null for that title/name. The available datasets are as follows:

**title.akas.tsv.gz** - Contains the following information for titles:

* titleId (string) - a tconst, an alphanumeric unique identifier of the title
* ordering (integer) – a number to uniquely identify rows for a given titleId
* title (string) – the localized title
* region (string) - the region for this version of the title
* language (string) - the language of the title
* types (array) - Enumerated set of attributes for this alternative title. One or more of the following: "alternative", "dvd", "festival", "tv", "video", "working", "original", "imdbDisplay". New values may be added in the future without warning
* attributes (array) - Additional terms to describe this alternative title, not enumerated
* isOriginalTitle (boolean) – 0: not original title; 1: original title

**title.basics.tsv.gz** - Contains the following information for titles:

* tconst (string) - alphanumeric unique identifier of the title
* titleType (string) – the type/format of the title (e.g. movie, short, tvseries, tvepisode, video, etc)
* primaryTitle (string) – the more popular title / the title used by the filmmakers on promotional materials at the point of release
* originalTitle (string) - original title, in the original language
* isAdult (boolean) - 0: non-adult title; 1: adult title
* startYear (YYYY) – represents the release year of a title. In the case of TV Series, it is the series start year
* endYear (YYYY) – TV Series end year. ‘\N’ for all other title types
* runtimeMinutes – primary runtime of the title, in minutes
* genres (string array) – includes up to three genres associated with the title

**title.ratings.tsv.gz** – Contains the IMDb rating and votes information for titles

* tconst (string) - alphanumeric unique identifier of the title
* averageRating – weighted average of all the individual user ratings
* numVotes - number of votes the title has received

## Data Cleaning

### Load Libraries

In [1]:
## General Libraries
import pandas as pd
import numpy as np
import warnings
import os
warnings.simplefilter('ignore')

## specifying data folder
FOLDER = "data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['basics.csv.gz',
 'aka.csv.gz',
 'ratings.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'mod',
 'backup_data']

### Loading DataFiles

In [2]:
%%time
title_basics = 'https://datasets.imdbws.com/title.basics.tsv.gz'
title_aka = 'https://datasets.imdbws.com/title.akas.tsv.gz'
title_ratings = 'https://datasets.imdbws.com/title.ratings.tsv.gz'

## loading urls into pandas dataframe
basics = pd.read_csv(title_basics, sep='\t', low_memory='False')
aka = pd.read_csv(title_aka, sep='\t', low_memory='False')
ratings = pd.read_csv(title_ratings, sep='\t', low_memory='False')


KeyboardInterrupt



In [3]:
print('basics = ', 'len:',len(basics))
display(basics.head())
print('\naka = ', 'len:',len(aka))
display(aka.head())
print('\nratings = ', 'len:',len(ratings))
display(ratings.head())


KeyboardInterrupt



#### Replacing \N with NaN

The data dictionary reports that the files shows null values as \N, this would be an issue so it must be changed to NaN values.

In [ ]:
## Replacing the \N values with NaN
basics.replace({'\\N': np.nan}, inplace=True)
aka.replace({'\\N': np.nan}, inplace=True)
ratings.replace({'\\N': np.nan}, inplace=True)

In [ ]:
print('basics')
display(basics.head())
print('\naka')
display(aka.head())
print('\nratings')
display(ratings.head())

#### Eliminate runtimeMinute = Null

In [ ]:
basics.dropna(subset=['runtimeMinutes', 'genres'], axis=0, inplace=True)

#### Keeping only titleType = movie

In [ ]:
basics = basics[basics['titleType'] == 'movie']

#### Keeping startYear between 2000-2021

In [ ]:
basics = basics[(basics['startYear'] >= '2000') & (basics['startYear'] < '2021')]

#### Dropping movies that contain documentaries

In [ ]:
## filtering movies that contain documentaries in genres
is_documentary = basics['genres'].str.contains('documentary', case=False)
##saving
basics = basics[~is_documentary]
## showing results
basics.head(10)

#### Keeping only US movies

In [ ]:
aka = aka[aka['region'] == 'US']

Applying to all other sets

In [ ]:
# Filtering basics
us_movies = basics['tconst'].isin(aka['titleId'])
# results
us_movies[:10]

In [ ]:
#Apply to the basics set
basics = basics[us_movies]

In [ ]:
## filtering ratings
us_ratings = ratings['tconst'].isin(aka['titleId'])
# results
us_ratings[:10]

In [ ]:
## Apply to ratings se
ratings = ratings[us_ratings]

## Saving DataFrames

In [ ]:
import os
os.makedirs('data/', exist_ok=True)
os.listdir('data/')

In [ ]:
## saving basics to compressed file
basics.to_csv("data/basics.csv.gz",compression='gzip',index=False)
## saving aka to compressed file
aka.to_csv("data/aka.csv.gz",compression='gzip',index=False)
## saving ratings to compressed file
ratings.to_csv("data/ratings.csv.gz",compression='gzip',index=False)

End